Import

In [1]:
import subprocess
import shlex
import os.path
import sys
import pickle
import datetime
import tweepy
import matplotlib.pyplot as plt 
import numpy as np
from scipy import spatial  
from sklearn.metrics import r2_score
import string
import numpy as np
import matplotlib.pyplot as plt
import collections
from itertools import tee, islice
from sklearn.decomposition import PCA
import math

Other Helper Functions

In [2]:
def get_list_of_users(tweets):

    list_of_user = []

    for tweet in tweets:
        user_id = tweet.user.id
        if user_id not in list_of_user:
            list_of_user.append(user_id)

    return list_of_user

Import Tweet and Related Data

In [ ]:
# Where you want to save the coverted files # TO BE REMOVED
test_directory = "" 

In [3]:
#Location of all the ".p" files containing tweets of friends
directory = "L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Python Tweets/" 

# Location of sentiment analysis file
sentiment_directory = "L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Python Tweets/Sentiment/"
analyzed_users = "L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Python Tweets/data_analyzed_1.p"

In [4]:
# Tweets output from the function, get_inital_tweets
initial_tweet_file = directory + 'user_tweet_large.p'
initial_tweets = pickle.load(open(initial_tweet_file, "rb"))
trending_tweet_file = directory + 'trending_tweets.p'
trending_tweets = pickle.load(open(trending_tweet_file, "rb"))

In [3]:
# Don't use this if using above
already_analyzed = pickle.load(open(analyzed_users, "rb"))
list_of_users = []

for uid in already_analyzed:
    list_of_users.append(uid)

Prototype and Exemplar Models - Helper Functions

In [3]:
# Based on code from COG260
# Helper method for similarity function
def similarity_function(feature_vector_0, feature_vector_1):
    euclidean_distance = spatial.distance.euclidean(feature_vector_0, feature_vector_1)
    temp_calc = euclidean_distance**2
    similarity = np.exp(0 - temp_calc)
    return similarity
    
    
# Helper method for examplar model
def categorize_with_exemplar(test_data_point, category_0, category_1, category_2):
    
    similarity_sum_cateogry_0 = []
    similarity_sum_cateogry_1 = []
    similarity_sum_cateogry_2 = []
    
    for object_0 in category_0:
        similarity_sum_cateogry_0.append(similarity_function(object_0, test_data_point))
        
    for object_1 in category_1:
        similarity_sum_cateogry_1.append(similarity_function(object_1, test_data_point))
        
    for object_2 in category_2:
        similarity_sum_cateogry_2.append(similarity_function(object_2, test_data_point))
        
    similarity_sum_cateogry_0 = np.sum(similarity_sum_cateogry_0)
    similarity_sum_cateogry_1 = np.sum(similarity_sum_cateogry_1)
    similarity_sum_cateogry_2 = np.sum(similarity_sum_cateogry_2)
    
    similarity_0 = similarity_sum_cateogry_0 / len(category_0)
    similarity_1 = similarity_sum_cateogry_1 / len(category_1)
    similarity_2 = similarity_sum_cateogry_2 / len(category_2)
    
    combined = [similarity_0, similarity_1, similarity_2]
    return_value = combined.index(max(combined))
    #print(combined)
    #print(return_value)
    
    return return_value

def categorize_with_2_exemplar(test_data_point, category_0, category_1):
    
    similarity_sum_cateogry_0 = []
    similarity_sum_cateogry_1 = []
    
    for object_0 in category_0:
        similarity_sum_cateogry_0.append(similarity_function(object_0, test_data_point))
        
    for object_1 in category_1:
        similarity_sum_cateogry_1.append(similarity_function(object_1, test_data_point))
        
    similarity_sum_cateogry_0 = np.sum(similarity_sum_cateogry_0)
    similarity_sum_cateogry_1 = np.sum(similarity_sum_cateogry_1)
    
    similarity_0 = similarity_sum_cateogry_0 / len(category_0)
    similarity_1 = similarity_sum_cateogry_1 / len(category_1)

    combined = [similarity_0, similarity_1]
    return_value = combined.index(max(combined))
    
    return return_value

In [4]:
# Helper method to get the prototype
def get_prototype(data_point):
        
    prototype = []
        
    for i in range(len(data_point[0, :])):
        prototype.append(np.mean(data_point[:,i]))
            
    return prototype

def categorize_with_prototype(test_data_point, category_0, category_1, category_2):
    
    cat_0_distance = spatial.distance.euclidean(test_data_point, category_0)
    cat_1_distance = spatial.distance.euclidean(test_data_point, category_1)
    cat_2_distance = spatial.distance.euclidean(test_data_point, category_2)
    
    combined = [cat_0_distance, cat_1_distance, cat_2_distance]
    return_value = combined.index(min(combined))
    
    return return_value

def categorize_with_2_prototype(test_data_point, category_0, category_1):
    
    cat_0_distance = spatial.distance.euclidean(test_data_point, category_0)
    cat_1_distance = spatial.distance.euclidean(test_data_point, category_1)
    
    combined = [cat_0_distance, cat_1_distance]
    return_value = combined.index(min(combined))
    
    return return_value

Prototype and Exemplar Model - Model Prep

In [5]:
sentiment_directory = "L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Python Tweets/Sentiment/"
analyzed_users = "L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Python Tweets/data_analyzed_1.p"
analyzed_users = "L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Python Tweets/data_analyzed_1.p"

positive_x = []
positive_y = []
positive_z = []

negative_x = []
negative_y = []
negative_z = []

neutral_x = []
neutral_y = []
neutral_z = []

positive_data_point = []
negative_data_point = []
neutral_data_point = []

already_analyzed = pickle.load(open(analyzed_users, "rb"))
list_of_users = []

for uid in already_analyzed:
    list_of_users.append(uid)

# Analyze all the  users in initial_tweets
for user in list_of_users:
    
    file_name = sentiment_directory + str(user) + '_sent.p'
    
    if os.path.exists(file_name):
        
        tweet_sentiment = pickle.load(open(file_name, "rb"))
        
        for tweet_id in tweet_sentiment:
            positive = 0
            negative = 0
            neutral = 0
            contain_pos = 0
            contain_neg = 0
            
            tweet_sent = tweet_sentiment[tweet_id]['combined']

            if 'friend' in tweet_sentiment[tweet_id]:
                for tweet in tweet_sentiment[tweet_id]['friend']:
                    friend_sentiment = tweet_sentiment[tweet_id]['friend'][tweet]['combined']
                    
                    if friend_sentiment > 0:
                        positive += 1
                    elif friend_sentiment < 0:
                        negative += 1
                    else:
                        neutral += 1
                        
                    friend_sentiment_pos = tweet_sentiment[tweet_id]['friend'][tweet]['sentiment'][0]
                    friend_sentiment_neg = tweet_sentiment[tweet_id]['friend'][tweet]['sentiment'][1]
                    
                    if friend_sentiment_pos > 2:
                        contain_pos += 1
                        
                    if friend_sentiment_neg < -1:
                        contain_pos += 1
                            
                total = positive + negative + neutral
                
                if total > 0:
                    positive_percent = positive/total
                    negative_percent = negative/total
                    neutral_percent = neutral/total
                    
                    cont_pos_percent = contain_pos/total
                    cont_neg_percent = contain_neg/total
                    
                    temp = [positive_percent, negative_percent, neutral_percent, cont_pos_percent, cont_neg_percent]
                    temp = np.asarray(temp)

                    if tweet_sent > 0:
                        positive_x.append(positive_percent)
                        positive_y.append(negative_percent)
                        positive_z.append(neutral_percent)

                        positive_data_point.append(temp)
                    elif tweet_sent < 0:
                        negative_x.append(positive_percent)
                        negative_y.append(negative_percent)
                        negative_z.append(neutral_percent) 

                        negative_data_point.append(temp)
                    else:
                        neutral_x.append(positive_percent)
                        neutral_y.append(negative_percent)
                        neutral_z.append(neutral_percent)  

                        neutral_data_point.append(temp)

positive_data_point = np.asarray(positive_data_point)
negative_data_point = np.asarray(negative_data_point)
neutral_data_point = np.asarray(neutral_data_point)

Prototype Model

In [6]:
# Get Prototype

positive_prototype = get_prototype(positive_data_point)
negative_prototype = get_prototype(negative_data_point)
neutral_prototype = get_prototype(neutral_data_point)

In [7]:
# Prep for Testing Model
result = {}

NZ_tweet_dir = predictive_model = "L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Partner Code/"
predictive_model = "L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Partner Code/Sentiment/"

#tweet_file_name = predictive_model  + 'user_tweet_large_NZ_1.p'
#predictive_user_tweets = pickle.load(open(tweet_file_name, "rb"))

trending_tweet_file_NZ = NZ_tweet_dir + 'trending_tweets_NZ.p'
trending_tweets_NZ = pickle.load(open(trending_tweet_file_NZ, "rb"))

list_of_users_NZ = get_list_of_users(trending_tweets_NZ)

analyzed_users_NZ = predictive_model  + 'data_analyzed_1.p'
already_analyzed_NZ = pickle.load(open(analyzed_users_NZ, "rb"))

for uid in already_analyzed_NZ:
    if uid not in list_of_users_NZ:
        list_of_users_NZ.append(uid)
        
print(len(list_of_users_NZ))

191


In [56]:
# Prototype - only positive and negative

total_count = 0
accurate = 0

p_total = 0
p_acc = 0

n_total = 0
n_acc = 0

ne_total = 0
ne_acc = 0

pos_total = 0
neg_total = 0

p_miss = 0
n_miss = 0

positive_total = 0
negative_total = 0
    
# Analyze all the  users in initial_tweets
for user in list_of_users_NZ:
    
    file_name = predictive_model + str(user) + '_sent.p'
    
    if os.path.exists(file_name):
        
        tweet_sentiment = pickle.load(open(file_name, "rb"))
        
        for tweet_id in tweet_sentiment:

            positive = 0
            negative = 0
            neutral = 0
            contain_pos = 0
            contain_neg = 0
            
            tweet_sent = tweet_sentiment[tweet_id]['combined']
            
            if tweet_sent != 0:

                if 'friend' in tweet_sentiment[tweet_id]:
                    for tweet in tweet_sentiment[tweet_id]['friend']:

                        friend_sentiment = tweet_sentiment[tweet_id]['friend'][tweet]['combined']
                        if friend_sentiment > 0:
                            positive += 1
                        elif friend_sentiment < 0:
                            negative += 1
                        else:
                            neutral += 1
                            
                        friend_sentiment_pos = tweet_sentiment[tweet_id]['friend'][tweet]['sentiment'][0]
                        friend_sentiment_neg = tweet_sentiment[tweet_id]['friend'][tweet]['sentiment'][1]
                    
                        if friend_sentiment_pos > 2:
                            contain_pos += 1
                        
                        if friend_sentiment_neg < -1:
                            contain_pos += 1
                            

                total = positive + negative + neutral
                
                if total > 0:
                    positve_percent = positive/total
                    negative_percent = negative/total
                    neutral_percent = neutral/total
                    
                    cont_pos_percent = contain_pos/total
                    cont_neg_percent = contain_neg/total
                    
                    temp = [positive_percent, negative_percent, neutral_percent, cont_pos_percent, cont_neg_percent]
                    temp = np.asarray(temp)

                    exemplar = categorize_with_2_prototype(temp, positive_prototype, negative_prototype) 
                    total_count += 1
                    
                    if exemplar == 0:
                        pos_total += 1
                        
                    if exemplar == 1:
                        neg_total += 1

                    if tweet_sent > 0:
                        p_total += 1
                        if exemplar == 0:
                            accurate += 1
                            p_acc += 1
                        else:
                            p_miss += 1
                    elif tweet_sent < 0:
                        n_total += 1
                        if exemplar == 1:
                            accurate += 1
                            n_acc += 1
                        else:
                            n_miss += 1

In [60]:
# Result for Prototype Model
print("Total Accurate " + str(accurate))
print("Total Count " + str(total_count))
print("Positive Percent Predicted " + str(p_acc/p_total))
print("Negative Percevent Predicted " + str(n_acc/n_total))
print("True Positive: " + str(p_acc/pos_total))
print("False Positive: " + str((pos_total - p_acc)/pos_total))
print("Missed Positive: " + str(p_miss))
print("True Negative: " + str(n_acc/neg_total))
print("False Negative: " + str((neg_total - n_acc)/neg_total))
print("Missed Negative: " + str(n_miss))
print("Positive Post " + str(p_total))
print("Negative Post " + str(n_total))
print("Positive Accurate Predict "+ str(p_acc))
print("Negative Accurate Predict "+ str(n_acc))
print("# of positive prediction " + str(pos_total))
print("# of negative prediction " + str(neg_total))
#print(ne_acc/ne_total)
#print(accurate/total_count)

Total Accurate 2017
Total Count 2581
Positive Percent Predicted 0.6361290322580645
Negative Percevent Predicted 1.0
True Positive: 1.0
False Positive: 0.0
Missed Positive: 564
True Negative: 0.6463949843260188
False Negative: 0.35360501567398117
Missed Negative: 0
Positive Post 1550
Negative Post 1031
Positive Accurate Predict 986
Negative Accurate Predict 1031
# of positive prediction 986
# of negative prediction 1595


In [8]:
# Prototype

total_count = 0
accurate = 0

p_total = 0
p_acc = 0

n_total = 0
n_acc = 0

ne_total = 0
ne_acc = 0

pos_total = 0
neg_total = 0
neu_total = 0

p_miss = 0
n_miss = 0
ne_miss = 0
    
# Analyze all the  users in initial_tweets
for user in list_of_users_NZ:
    
    file_name = predictive_model + str(user) + '_sent.p'
    
    if os.path.exists(file_name):
        
        tweet_sentiment = pickle.load(open(file_name, "rb"))
        
        for tweet_id in tweet_sentiment:

            positive = 0
            negative = 0
            neutral = 0
            contain_pos = 0
            contain_neg = 0
            
            tweet_sent = tweet_sentiment[tweet_id]['combined']
            

            if 'friend' in tweet_sentiment[tweet_id]:
                for tweet in tweet_sentiment[tweet_id]['friend']:

                    friend_sentiment = tweet_sentiment[tweet_id]['friend'][tweet]['combined']
                    if friend_sentiment > 0:
                        positive += 1
                    elif friend_sentiment < 0:
                        negative += 1
                    else:
                        neutral += 1
                            
                    friend_sentiment_pos = tweet_sentiment[tweet_id]['friend'][tweet]['sentiment'][0]
                    friend_sentiment_neg = tweet_sentiment[tweet_id]['friend'][tweet]['sentiment'][1]
                    
                    if friend_sentiment_pos > 2:
                        contain_pos += 1
                        
                    if friend_sentiment_neg < -1:
                        contain_pos += 1
                            

            total = positive + negative + neutral
            if total > 0:
                positve_percent = positive/total
                negative_percent = negative/total
                neutral_percent = neutral/total
                    
                cont_pos_percent = contain_pos/total
                cont_neg_percent = contain_neg/total
                    
                temp = [positive_percent, negative_percent, neutral_percent, cont_pos_percent, cont_neg_percent]
                temp = np.asarray(temp)

                exemplar = categorize_with_prototype(temp, positive_prototype, negative_prototype, neutral_prototype) 
                total_count += 1
                    
                if exemplar == 0:
                    pos_total += 1
                        
                if exemplar == 1:
                    neg_total += 1
                    
                if exemplar == 2:
                    neu_total += 1

                    if tweet_sent > 0:
                        p_total += 1
                        if exemplar == 0:
                            accurate += 1
                            p_acc += 1
                        else:
                            p_miss += 1
                    elif tweet_sent < 0:
                        n_total += 1
                        if exemplar == 1:
                            accurate += 1
                            n_acc += 1
                        else:
                            n_miss += 1
                    else:
                        ne_total += 1
                        if exemplar == 2:
                            accurate += 1
                            ne_acc += 1
                        else:
                            ne_miss += 1

In [12]:
# Result for Prototype Model
print("Total Accurate " + str(accurate))
print("Total Count " + str(total_count))
#print("Positive Percent Predicted " + str(p_acc/p_total))
#print("Negative Percevent Predicted " + str(n_acc/n_total))
print("Neutral Percevent Predicted " + str(ne_acc/ne_total))
#print("True Positive: " + str(p_acc/pos_total))
#print("False Positive: " + str((pos_total - p_acc)/pos_total))
print("Missed Positive: " + str(p_miss))
#print("True Negative: " + str(n_acc/neg_total))
#print("False Negative: " + str((neg_total - n_acc)/neg_total))
print("Missed Negative: " + str(n_miss))
#print("True Neutral: " + str(ne_acc/neu_total))
#print("False Neutral: " + str((neu_total - ne_acc)/neu_total))
print("Missed Neutral: " + str(ne_miss))
print("Positive Post " + str(p_total))
print("Negative Post " + str(n_total))
print("Neutral Post " + str(ne_total))
print("Positive Accurate Predict "+ str(p_acc))
print("Negative Accurate Predict "+ str(n_acc))
print("Neutral Accurate Predict "+ str(ne_acc))
print("# of positive prediction " + str(pos_total))
print("# of negative prediction " + str(neg_total))
print("# of neutral prediction " + str(neu_total))
#print(ne_acc/ne_total)
#print(accurate/total_count)

Total Accurate 2283
Total Count 5164
Neutral Percevent Predicted 1.0
Missed Positive: 986
Missed Negative: 0
Missed Neutral: 0
Positive Post 986
Negative Post 0
Neutral Post 2283
Positive Accurate Predict 0
Negative Accurate Predict 0
Neutral Accurate Predict 2283
# of positive prediction 0
# of negative prediction 1895
# of neutral prediction 3269


Exemplar Model

In [61]:
total_count = 0
accurate = 0

p_total = 0
p_acc = 0

n_total = 0
n_acc = 0

ne_total = 0
ne_acc = 0

pos_total = 0
neg_total = 0
    
# Analyze all the  users in initial_tweets
for user in list_of_users_NZ:
    
    file_name = predictive_model + str(user) + '_sent.p'
    
    if os.path.exists(file_name):
        
        tweet_sentiment = pickle.load(open(file_name, "rb"))
        
        for tweet_id in tweet_sentiment:

            positive = 0
            negative = 0
            neutral = 0
            contain_pos = 0
            contain_neg = 0
            
            tweet_sent = tweet_sentiment[tweet_id]['combined']
            
            if tweet_sent != 0:

                if 'friend' in tweet_sentiment[tweet_id]:
                    for tweet in tweet_sentiment[tweet_id]['friend']:

                        friend_sentiment = tweet_sentiment[tweet_id]['friend'][tweet]['combined']
                        if friend_sentiment > 0:
                            positive += 1
                        elif friend_sentiment < 0:
                            negative += 1
                        else:
                            neutral += 1
                            
                        friend_sentiment_pos = tweet_sentiment[tweet_id]['friend'][tweet]['sentiment'][0]
                        friend_sentiment_neg = tweet_sentiment[tweet_id]['friend'][tweet]['sentiment'][1]
                    
                        if friend_sentiment_pos > 2:
                            contain_pos += 1
                        
                        if friend_sentiment_neg < -1:
                            contain_pos += 1
                            

                total = positive + negative + neutral
                if total > 0:
                    positve_percent = positive/total
                    negative_percent = negative/total
                    neutral_percent = neutral/total
                    
                    cont_pos_percent = contain_pos/total
                    cont_neg_percent = contain_neg/total
                    
                    temp = [positive_percent, negative_percent, neutral_percent, cont_pos_percent, cont_neg_percent]
                    temp = np.asarray(temp)

                    exemplar = categorize_with_2_exemplar(temp, positive_data_point, negative_data_point) 
                    total_count += 1
                    
                    if exemplar == 0:
                        pos_total += 1
                        
                    if exemplar == 1:
                        neg_total += 1

                    if tweet_sent > 0:
                        p_total += 1
                        if exemplar == 0:
                            accurate += 1
                            p_acc += 1
                    elif tweet_sent < 0:
                        n_total += 1
                        if exemplar == 1:
                            accurate += 1
                            n_acc += 1

In [62]:
# Result for Examplar Model
print("Total Accurate " + str(accurate))
print("Total Count " + str(total_count))
print("Positive Percent Predicted " + str(p_acc/p_total))
print("Negative Percevent Predicted " + str(n_acc/n_total))
print("True Positive: " + str(p_acc/pos_total))
print("False Positive: " + str((pos_total - p_acc)/pos_total))
print("Missed Positive: " + str(p_miss))
print("True Negative: " + str(n_acc/neg_total))
print("False Negative: " + str((neg_total - n_acc)/neg_total))
print("Missed Negative: " + str(n_miss))
print("Positive Post " + str(p_total))
print("Negative Post " + str(n_total))
print("Positive Accurate Predict "+ str(p_acc))
print("Negative Accurate Predict "+ str(n_acc))
print("# of positive prediction " + str(pos_total))
print("# of negative prediction " + str(neg_total))
#print(ne_acc/ne_total)
#print(accurate/total_count)

Total Accurate 2017
Total Count 2581
Positive Percent Predicted 0.6361290322580645
Negative Percevent Predicted 1.0
True Positive: 1.0
False Positive: 0.0
Missed Positive: 564
True Negative: 0.6463949843260188
False Negative: 0.35360501567398117
Missed Negative: 0
Positive Post 1550
Negative Post 1031
Positive Accurate Predict 986
Negative Accurate Predict 1031
# of positive prediction 986
# of negative prediction 1595
